<div style="max-width:40em">
    Create plots of temperature and winds at Troll and TAF.<br><br>
    To create a 4-panel figure in portrait orientation with labels in Norwegian, for the previous full calendar month, just select Run above and then Run All Cells.<br><br>
    You can also edit the last block of code below to change the time period or figure setup/language. (Run > Run All Cells after editing.)<br><br>
    Once run, the figure should appear below the last block of code after a minute (longer if you choose a long time period). You can then right click the displayed
    figure and select Copy Output to Clipboard or right click and select Create New View for Output, then in the new view right click and Save Image As. A PNG file
    with the figure will also appear in the directory display on the left; you can open that and Save Image As or right click that and Download.<br><br>
</div>

<div style="max-width:40em">
    <br><br>Import required packages
</div>

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
import requests
from io import StringIO
import warnings

<div style="max-width:50em">
    Define function to download data from api.npolar.no based on the dates in the bottom cell
</div>

In [ ]:
def get_data(stn_id='', start_date=None, end_date=None):
    if stn_id.casefold() == 'troll'.casefold():
        stn = 'kldata/nationalnr=99990/type=504'
    elif stn_id.casefold() == 'TAF'.casefold():
        stn = 'kldata/nationalnr=92100/type=504'
    else:
        raise ValueError('get_data(stn_id, start_date, end_date):\n stn_id is required and must match ''Troll'' or ''TAF'' (case insensitive)')
    if (start_date is None and end_date is None):
        d_start = pd.Timestamp.now().normalize() - pd.offsets.MonthEnd() - pd.offsets.MonthBegin()
        d_end = (pd.Timestamp.now().normalize() + pd.offsets.Day(1)) - pd.offsets.MonthBegin() - pd.Timedelta(milliseconds=1) #add 1 day so -MonthBegin moves to start of current month also on the first day of the month
    elif (start_date is not None and end_date is not None):
        try:
            d_start = pd.to_datetime(start_date)
            d_end = pd.to_datetime(end_date)
        except:
            raise ValueError('get_data(stn_id, start_date, end_date):\n start_date and end_date must convert to datetime with pd.to_datetime()')
        else:
            if d_start >= d_end or d_start >= pd.Timestamp.now():
                raise ValueError('get_data(stn_id, start_date, end_date):\n start_date must be before end_date and current time')
    else:
        if end_date is None:
            try:
                d_start = pd.to_datetime(start_date)
            except:
                raise ValueError('get_data(stn_id, start_date):\n start_date must convert to datetime with pd.to_datetime()')
            else:
                if d_start >= pd.Timestamp.now():
                    raise ValueError('get_data(stn_id, start_date):\n start_date must be before current time')
            d_end = pd.Timestamp.now()
        elif start_date is None:
            try:
                d_end = pd.to_datetime(end_date)
            except:
                raise ValueError('get_data(stn_id, end_date=''date_str''):\n end_date must convert to datetime with pd.to_datetime()')
            else:
                if d_end >= pd.Timestamp.now():
                    raise ValueError('get_data(stn_id, end_date=''date_str''):\n end_date must be before current time')
            d_start = d_end - pd.Timedelta(weeks=4)
    
    var = 'DD0_6,FF0_6,FG0_6,TA0_0,DD0_0,FF0_0,FG0_0'
    url = 'https://api.npolar.no/weather/troll/'
    parameters = {
        'q': '',
        'format': 'json',
        'filter-station_id': stn,
        'filter-measured': d_start.strftime('%Y-%m-%dT%H:%M:%S')+'..'+d_end.strftime('%Y-%m-%dT%H:%M:%S'),
        'fields': 'measured,'+var,
        'limit': '14400',
        'start': '0'
    }
    
    r = requests.get(url, params=parameters)
            
    data = pd.read_json(StringIO(r.text),orient='records')
    d = pd.DataFrame(data['feed']['entries'])
    d.measured = pd.to_datetime(d.measured)
    d = d.set_index('measured')
    
    while data['feed']['list']['next']:
        r = requests.get(data['feed']['list']['next'])
        data = pd.read_json(StringIO(r.text),orient='records')
        d1 = pd.DataFrame(data['feed']['entries'])
        d1.measured = pd.to_datetime(d1.measured)
        d1 = d1.set_index('measured')
        d = pd.concat([d,d1],sort=False)
    d[d==-99.9] = np.nan

    d.rename(columns={'FF0_6': 'FF0_0', 'FG0_6': 'FG0_0', 'DD0_6': 'DD0_0'}, inplace=True)
    d = d.sort_index()
    
    return d

<div style="max-width:40em">
    Define function to return hourly averages of observations, including an hourly vector-mean wind speed and direction. Also return the maximum gust in each 6-hour period.
</div>

In [ ]:
def resample_df(d):
    d['u'] = -d.FF0_0 * np.sin(np.pi * d.DD0_0 / 180)
    d['v'] = -d.FF0_0 * np.cos(np.pi * d.DD0_0 / 180)
    
    d_avg = d.resample('1h').mean()
    d_avg['sp'] = np.sqrt(d_avg.u*d_avg.u + d_avg.v*d_avg.v)
    d_avg['dir'] = 180 + (180/np.pi) * np.arctan2(d_avg.u,d_avg.v)
    
    d_avg['maxG'] = np.nan
    d_avg['maxG'] = d.FG0_0.resample('6h').max()

    return d_avg

<div style="max-width:40em">
    Define function to plot temperature at TAF and Troll in one panel and winds at TAF and Troll in a second panel, using different colors for TAF and Troll.<br><br>
    In addition to the data from the two locations, a third input may be included: 'eng' or 'nor', to label the figures in English or Norwegian.
</div>

In [ ]:
def plot_data2(dfTroll, dfTAF, lang='eng'):
    fs = 20 #font size, top title fs*1.25
    if (lang!='nor' and lang!='eng'):
        raise ValueError('plot_data(dfTroll, dfTAF, lang):\n lang must be ''nor'' or ''eng'', case sensitive')
    cTroll = 'peru'
    cTAF = 'lightskyblue'
    
    #English and Norwegian strings for labelling and naming the figure
    Tlabel  = {'nor': 'lufttemperatur', 'eng': 'air temperature'}
    spLabel = {'nor': 'vindhastighet', 'eng': 'wind speed'}
    gLabel  = {'nor': 'høyeste vindkast', 'eng': 'highest gust'}
    xLabel  = {'nor': 'Dato', 'eng': 'Date'}
    yLabel1 = {'nor': 'Temperatur (°C)', 'eng': 'Temperature (°C)'}
    yLabel2 = {'nor': 'Vindhastighet (m/s)', 'eng': 'Wind speed (m/s)'}
    title   = {'nor': 'Vær på Troll ', 'eng': 'Weather at Troll '}
    weather = {'nor': 'vær_', 'eng': 'weather_'}
    
    #Create empty figure with subplots
    f, (a1, a2) = plt.subplots(2, 1, figsize=(16,14), sharex=True)

    #Get hourly averaged data and 6-h gusts
    dfTroll = resample_df(dfTroll)
    dfTAF = resample_df(dfTAF)
    
    #Plot all time series
    a1.plot(dfTroll.index+pd.DateOffset(minutes=30), dfTroll.TA0_0, color=cTroll, label='Troll '+Tlabel[lang], linewidth='2')
    a1.plot(dfTAF.index+pd.DateOffset(minutes=30), dfTAF.TA0_0, color=cTAF, label='TAF '+Tlabel[lang], linewidth='2')
    a2.plot(dfTroll.index+pd.DateOffset(minutes=30), dfTroll.sp, color=cTroll, label='Troll '+spLabel[lang], linewidth='2')
    a2.plot(dfTroll.index+pd.DateOffset(hours=3), dfTroll.maxG, '.', color=cTroll, label='Troll '+gLabel[lang], markersize=8)
    a2.plot(dfTAF.index+pd.DateOffset(minutes=30), dfTAF.sp, color=cTAF, label='TAF '+spLabel[lang], linewidth='2')
    a2.plot(dfTAF.index+pd.DateOffset(hours=3), dfTAF.maxG, '.', color=cTAF, label='TAF '+gLabel[lang], markersize=8)
    
    #Start and end times of plotting period
    s = min(dfTroll.index.min(), dfTAF.index.min())
    e = max(dfTroll.index.max(), dfTAF.index.max())
    length = e - s
    #File name for saving figure, overridden below for case of one full calendar month
    saveFile = 'TrollTAF_' + weather[lang] + s.strftime('%Y-%m-%d_') + e.strftime('%Y-%m-%d.png')
    #Set up x tick labelling based on length of plotted period and includsion of month/year shifts
    #Over 6 months: major ticks at start of month, minor on 11 and 21 of month
    if length > pd.Timedelta(days=183):
        majLoc = 'mdates.MonthLocator(interval=1, bymonthday=1)'
        minLoc = 'mdates.DayLocator(bymonthday=[11, 21])'
        #Figure title includes year only on end time if start and end are in the same year (Continues below)
        if s.year==e.year:
            #Figure title includes day of month unless both start and end are on the first of the month (Continues below)
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' – ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #Two to six months: major ticks on the 1st and 15th, minor ticks every second day
    elif length > pd.Timedelta(days=62):
        majLoc = 'mdates.DayLocator(bymonthday=[1, 15])'
        minLoc = 'mdates.DayLocator(bymonthday=np.arange(3, 31, 2))'
        if s.year==e.year:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' – ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #42 to 62 days: major ticks on 1st, 11th and 21st of the month, minor ticks every day
    elif length > pd.Timedelta(days=41):
        majLoc = 'mdates.DayLocator(bymonthday=[1, 11, 21])'
        minLoc = 'mdates.DayLocator(interval=1)'
        if s.year==e.year:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' – ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #22 to 41 days: major ticks every fifth day, minor ticks every day
    elif length > pd.Timedelta(days=21):
        majLoc = 'mdates.DayLocator(bymonthday=[1, 6, 11, 16, 21, 26])'
        minLoc = 'mdates.DayLocator(interval=1)'
        if s.year==e.year:
            #Single calendar month: figure title and file name include only month and year
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(e.year)
                saveFile = 'TrollTAF_' + weather[lang] + s.strftime('%Y-%m.png')
            #Figure title includes month name only in end time if start and end are in the same calendar month (Continues below)
            else:
                if s.month==e.month:
                    figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
                else:
                    figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                             str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #8 to 21 days: major ticks every day, minor ticks every 6 hours
    elif length > pd.Timedelta(days=7):
        majLoc = 'mdates.DayLocator(bymonthday=np.arange(1, 32))'
        minLoc = 'mdates.HourLocator(byhour=[0, 6, 12, 18])'
        if s.year==e.year:
            if s.month==e.month:
                figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #Four to seven days: major ticks every day, minor ticks every two hours
    elif length > pd.Timedelta(days=3):
        majLoc = 'mdates.DayLocator()'
        minLoc = 'mdates.HourLocator(byhour=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22])'
        if s.year==e.year:
            if s.month==e.month:
                figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #Up to three days: major ticks every six hours, minor ticks every hour
    else:
        majLoc = 'mdates.HourLocator(byhour=[0, 6, 12, 18, 24])'
        minLoc = 'mdates.HourLocator()'
        if s.year==e.year:
            if s.month==e.month:
                figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    
    #Set up ticks, labels, etc on top panel
    a1.minorticks_on()
    a1.tick_params(labelbottom=True)
    a1.tick_params(which='minor', length=0)
    a1.grid(which='both', color='gainsboro', linestyle='-', linewidth=1)
    a1.grid(color='k', linestyle='--')
    a1.tick_params(axis='both', labelsize=fs)
    a1.set_ylabel(yLabel1[lang], fontsize=fs)
    a1.legend(fontsize=fs)
    
    #Set up ticks, labels, etc on bottom panel
    a2.minorticks_on()
    a2.xaxis.set_major_locator(eval(majLoc))
    a2.xaxis.set_major_formatter(mdates.ConciseDateFormatter(eval(majLoc)))
    a2.xaxis.set_minor_locator(eval(minLoc))
    a2.tick_params(which='minor', length=0)
    a2.grid(which='both', color='gainsboro', linestyle='-', linewidth=1)
    a2.grid(color='k', linestyle='--')
    a2.set_xlabel(xLabel[lang], fontsize=fs)
    a2.tick_params(axis='both', labelsize=fs)
    a2.set_ylabel(yLabel2[lang], fontsize=fs)
    a2.legend(fontsize=fs)
    
    #Change month names and abbreviations in the figure title and x labels to Norwegian, if lang='nor'.
    #Using locale does not work in environments without the Norwegian locale installed.
    if lang=='nor':
        month_names = {'January': 'januar', 'February': 'februar', 'March': 'mars', 'April': 'april',
                       'May': 'mai', 'June': 'juni', 'July': 'juli', 'August': 'august', 'September': 'september',
                       'October': 'oktober', 'November': 'november', 'December': 'desember'}
        for en, no in month_names.items():
            figTitle = figTitle.replace(en, no)
        mon_names = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr',
                     'May': 'mai', 'Jun': 'jun', 'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep',
                     'Oct': 'okt', 'Nov': 'nov', 'Dec': 'des'}
        tick_labels = a2.get_xticklabels()
        tick_labels = [mon_names.get(tl.get_text(), tl.get_text()) for tl in tick_labels]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            a2.set_xticklabels(tick_labels)
    
    #Add overall figure title above top panel
    f.suptitle(figTitle, fontsize=fs*1.25)
    #Adjust formatting, save figure
    plt.tight_layout()
    plt.show()
    f.savefig(saveFile)
    
    #Return figure and axes handles
    return f, a1, a2

<div style="max-width:40em">
    Define function to plot temperature at TAF, temperature at Troll, winds at TAF and winds at Troll in four separate panels.<br><br>
    In addition to the data from the two locations, a third input may be included: 'eng' or 'nor', to label the figures in English or Norwegian. A fourth input, 'portrait' or 'landscape' arranges the panels in one column and four rows or two columns and two rows.
</div>

In [ ]:
def plot_data4(dfTroll, dfTAF, lang='eng', orient='portrait'):
    fs = 20 #font size, top title fs*1.25
    if (lang!='nor' and lang!='eng'):
        raise ValueError('plot_data(dfTroll, dfTAF, lang):\n lang must be ''nor'' or ''eng'', case sensitive')
    if (orient!='portrait' and orient!='landscape'):
        raise ValueError('plot_data(dfTroll, dfTAF, lang, orient):\n orient must be ''portrait'' or ''landscape'', case sensitive')
    #Colors to use for plotted data
    c1 = 'peru'
    c2 = 'firebrick'
    
    #Setup (size, number of rows/columns, legend position) for portrait and landscape orientation
    sz      = {'portrait': (15, 15*297/210), 'landscape': (15*297/210, 15)}
    nrc     = {'portrait': [4, 1], 'landscape': [2, 2]}
    leg     = {'portrait': (0, 1.15), 'landscape': (0, 1.1)}
    #English and Norwegian strings for labelling and naming the figure
    Tlabel  = {'nor': 'lufttemperatur', 'eng': 'air temperature'}
    spLabel = {'nor': 'vindhastighet', 'eng': 'wind speed'}
    gLabel  = {'nor': 'høyeste vindkast', 'eng': 'highest gust'}
    xLabel  = {'nor': 'Dato', 'eng': 'Date'}
    yLabel1 = {'nor': 'Temperatur (°C)', 'eng': 'Temperature (°C)'}
    yLabel2 = {'nor': 'Vindhastighet (m/s)', 'eng': 'Wind speed (m/s)'}
    title   = {'nor': 'Vær på Troll ', 'eng': 'Weather at Troll '}
    weather = {'nor': 'vær_', 'eng': 'weather_'}
    
    #Create empty figure with subplots
    if orient=='portrait':
        f, (a1, a2, a3, a4) = plt.subplots(4, 1, figsize=sz[orient], sharex=True, sharey=False)
    elif orient=='landscape':
        f, ((a1, a3), (a2, a4)) = plt.subplots(2, 2, figsize=sz[orient], sharex=True, sharey='row')

    #Get hourly averaged data and 6-h gusts
    dfTroll = resample_df(dfTroll)
    dfTAF = resample_df(dfTAF)
    
    #Plot all time series
    a1.plot(dfTroll.index+pd.DateOffset(minutes=30), dfTroll.TA0_0, color=c1, label='Troll '+Tlabel[lang], linewidth='2')
    a3.plot(dfTAF.index+pd.DateOffset(minutes=30), dfTAF.TA0_0, color=c1, label='TAF '+Tlabel[lang], linewidth='2')
    a2.plot(dfTroll.index+pd.DateOffset(minutes=30), dfTroll.sp, color=c1, label='Troll '+spLabel[lang], linewidth='2')
    a2.plot(dfTroll.index+pd.DateOffset(hours=3), dfTroll.maxG, '.', color=c2, label='Troll '+gLabel[lang], markersize=10)
    a4.plot(dfTAF.index+pd.DateOffset(minutes=30), dfTAF.sp, color=c1, label='TAF '+spLabel[lang], linewidth='2')
    a4.plot(dfTAF.index+pd.DateOffset(hours=3), dfTAF.maxG, '.', color=c2, label='TAF '+gLabel[lang], markersize=10)
    
    #Start and end times of plotting period
    s = min(dfTroll.index.min(), dfTAF.index.min())
    e = max(dfTroll.index.max(), dfTAF.index.max())
    length = e - s
    #File name for saving figure, overridden below for case of one full calendar month
    saveFile = 'TrollTAF_' + weather[lang] + s.strftime('%Y-%m-%d_') + e.strftime('%Y-%m-%d.png')
    #Set up x tick labelling based on length of plotted period and includsion of month/year shifts
    #Over 6 months: major ticks at start of month, minor on 11 and 21 of month
    if length > pd.Timedelta(days=183):
        majLoc = 'mdates.MonthLocator(interval=1, bymonthday=1)'
        minLoc = 'mdates.DayLocator(bymonthday=[11, 21])'
        #Figure title includes year only on end time if start and end are in the same year (Continues below)
        if s.year==e.year:
            #Figure title includes day of month unless both start and end are on the first of the month (Continues below)
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' – ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #Two to six months: major ticks on the 1st and 15th, minor ticks every second day
    elif length > pd.Timedelta(days=62):
        majLoc = 'mdates.DayLocator(bymonthday=[1, 15])'
        minLoc = 'mdates.DayLocator(bymonthday=np.arange(3, 31, 2))'
        if s.year==e.year:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' – ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #42 to 62 days: major ticks on 1st, 11th and 21st of the month, minor ticks every day
    elif length > pd.Timedelta(days=41):
        majLoc = 'mdates.DayLocator(bymonthday=[1, 11, 21])'
        minLoc = 'mdates.DayLocator(interval=1)'
        if s.year==e.year:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' – ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #22 to 41 days: major ticks every fifth day, minor ticks every day
    elif length > pd.Timedelta(days=21):
        majLoc = 'mdates.DayLocator(bymonthday=[1, 6, 11, 16, 21, 26])'
        minLoc = 'mdates.DayLocator(interval=1)'
        if s.year==e.year:
            #Single calendar month: figure title and file name include only month and year
            if s.is_month_start & e.is_month_end:
                figTitle = title[lang] + s.strftime('%B') + ' ' + str(e.year)
                saveFile = 'TrollTAF_' + weather[lang] + s.strftime('%Y-%m.png')
            #Figure title includes month name only in end time if start and end are in the same calendar month (Continues below)
            else:
                if s.month==e.month:
                    figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
                else:
                    figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                             str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #8 to 21 days: major ticks every day, minor ticks every 6 hours
    elif length > pd.Timedelta(days=7):
        majLoc = 'mdates.DayLocator(bymonthday=np.arange(1, 32))'
        minLoc = 'mdates.HourLocator(byhour=[0, 6, 12, 18])'
        if s.year==e.year:
            if s.month==e.month:
                figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #Four to seven days: major ticks every day, minor ticks every two hours
    elif length > pd.Timedelta(days=3):
        majLoc = 'mdates.DayLocator()'
        minLoc = 'mdates.HourLocator(byhour=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22])'
        if s.year==e.year:
            if s.month==e.month:
                figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    #Up to three days: major ticks every six hours, minor ticks every hour
    else:
        majLoc = 'mdates.HourLocator(byhour=[0, 6, 12, 18, 24])'
        minLoc = 'mdates.HourLocator()'
        if s.year==e.year:
            if s.month==e.month:
                figTitle = title[lang] + str(s.day) + ' – ' + str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
            else:
                figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' – ' + \
                                         str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
        else:
            figTitle = title[lang] + str(s.day) + ' ' + s.strftime('%B') + ' ' + str(s.year) + ' – ' + \
                                     str(e.day) + ' ' + e.strftime('%B') + ' ' + str(e.year)
    
    #Set up ticks, labels, etc on each panel
    a1.minorticks_on()
    a1.tick_params(labelbottom=True)
    a1.tick_params(which='minor', length=0)
    a1.grid(which='both', color='gainsboro', linestyle='-', linewidth=1)
    a1.grid(color='k', linestyle='--')
    a1.tick_params(axis='both', labelsize=fs)
    a1.set_ylabel(yLabel1[lang], fontsize=fs)
    a1.legend(fontsize=fs, loc='upper left', bbox_to_anchor=leg[orient], framealpha=0.7)
    
    a2.minorticks_on()
    a2.tick_params(labelbottom=True)
    a2.tick_params(which='minor', length=0)
    a2.grid(which='both', color='gainsboro', linestyle='-', linewidth=1)
    a2.grid(color='k', linestyle='--')
    if orient=='landscape':
        a2.set_xlabel(xLabel[lang], fontsize=fs)
    a2.tick_params(axis='both', labelsize=fs)
    a2.set_ylabel(yLabel2[lang], fontsize=fs)
    a2.legend(fontsize=fs, loc='upper left', bbox_to_anchor=leg[orient], framealpha=0.7, ncol=2)
    
    a3.minorticks_on()
    a3.tick_params(labelbottom=True)
    a3.tick_params(which='minor', length=0)
    a3.grid(which='both', color='gainsboro', linestyle='-', linewidth=1)
    a3.grid(color='k', linestyle='--')
    a3.tick_params(axis='both', labelsize=fs)
    if orient=='portrait':
        a3.set_ylabel(yLabel1[lang], fontsize=fs)
    a3.legend(fontsize=fs, loc='upper left', bbox_to_anchor=leg[orient], framealpha=0.7)
    
    a4.minorticks_on()
    a4.xaxis.set_major_locator(eval(majLoc))
    a4.xaxis.set_major_formatter(mdates.ConciseDateFormatter(eval(majLoc)))
    a4.xaxis.set_minor_locator(eval(minLoc))
    a4.tick_params(which='minor', length=0)
    a4.grid(which='both', color='gainsboro', linestyle='-', linewidth=1)
    a4.grid(color='k', linestyle='--')
    a4.set_xlabel(xLabel[lang], fontsize=fs)
    a4.tick_params(axis='both', labelsize=fs)
    if orient=='portrait':
        a4.set_ylabel(yLabel2[lang], fontsize=fs)
    a4.legend(fontsize=fs, loc='upper left', bbox_to_anchor=leg[orient], framealpha=0.7, ncol=2)
    
    #Change month names and abbreviations in the figure title and x labels to Norwegian, if lang='nor'.
    #Using locale does not work in environments without the Norwegian locale installed.
    if lang=='nor':
        month_names = {'January': 'januar', 'February': 'februar', 'March': 'mars', 'April': 'april',
                       'May': 'mai', 'June': 'juni', 'July': 'juli', 'August': 'august', 'September': 'september',
                       'October': 'oktober', 'November': 'november', 'December': 'desember'}
        for en, no in month_names.items():
            figTitle = figTitle.replace(en, no)
        mon_names = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr',
                     'May': 'mai', 'Jun': 'jun', 'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep',
                     'Oct': 'okt', 'Nov': 'nov', 'Dec': 'des'}
        tick_labels = a4.get_xticklabels()
        tick_labels = [mon_names.get(tl.get_text(), tl.get_text()) for tl in tick_labels]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            a4.set_xticklabels(tick_labels)
        
    #Add overall figure title above top panels
    f.suptitle(figTitle+'\n', fontsize=fs*1.25)
    #Adjust formatting, save figure
    plt.tight_layout()
    plt.show()
    f.savefig(saveFile)
    
    #Return figure and axes handles
    return f, a1, a2, a3, a4

<div style="max-width:40em">
    Get desired data (previous month or start and end dates).<br><br>
    Plot data with desired formatting and labelling.
</div>

In [ ]:
#### Use these two lines to get the previous full calendar month
troll = get_data('Troll')
taf   = get_data('TAF')
#### OR these four lines to get an arbitrary time period
#start_date = '2020-05-01' #yyyy-mm-dd, full day is included, or yyyy-mm-ddTHH:MM:SS
#end_date   = '2020-08-01' #yyyy-mm-dd, stops at start of day, or yyyy-mm-ddTHH:MM:SS
#troll = get_data('Troll', start_date, end_date)
#taf = get_data('TAF', start_date, end_date)

#### Plot 4 panels stacked in portrait orientation, Norwegian text
fig, ax1, ax2, ax3, ax4 = plot_data4(troll, taf, 'nor', 'portrait')
#### Plot 4 panels 2x2 in landscape orientation, English text
#fig, ax1, ax2, ax3, ax4 = plot_data4(troll, taf, 'eng', 'landscape')
#### Plot 2 panels, Norwegian text
#fig, ax1, ax2 = plot_data2(troll, taf, 'nor')